In [ ]:
import pandas as pd
import os
import sys
from pathlib import Path  # Melhor forma para trabalhar com caminhos
from unidecode import unidecode


### Convertendo xlsx para csv


In [ ]:
pasta_origem = r'E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_02_DATASETS\datasets_original'
pasta_destino = r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_03_ANALISE_EXPLORATORIA\_03_02_analises_01"

for arquivo in os.listdir(pasta_origem):
    if arquivo.endswith('.xlsx'):
        entrada = os.path.join(pasta_origem, arquivo)
        saida = os.path.join(pasta_destino, arquivo.replace('.xlsx', '.csv'))
        try:
            df = pd.read_excel(entrada, header=1, engine='openpyxl')
            df.to_csv(saida, index=False)
            print(f"Convertido: {arquivo}")
        except Exception as e:
            print(f"Erro ao converter {arquivo}: {e}")


### Pivotando arquivos CSV

In [ ]:


def process_files(input_folder, output_folder):
    """Processa todos os arquivos CSV na pasta de entrada e salva na pasta de saída"""
    # Garante que a pasta de saída existe
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    

    for arquivo in os.listdir(input_folder):
        if arquivo.endswith('.csv') or arquivo.endswith('.xlsx'):
            try:
                nome = os.path.splitext(arquivo)[0]
                file_path = os.path.join(input_folder, arquivo)
                #trasnformar arquivo para csv e retirar a prima linha
                if arquivo.endswith('.xlsx'):
                    df = pd.read_excel(file_path, header=1)
                    #cria um  nome temporário para o CSV
                    temp_csv_path = os.path.join(input_folder, f"temp_{nome}.csv")
                    df.to_csv(temp_csv_path, index=False, encoding='utf-8')
                    file_path = temp_csv_path
                # Ler o CSV
                df = pd.read_csv(file_path, encoding='utf-8')
                
                # Transformar de wide para long
                df_long = df.melt(
                    id_vars=['Código da Conta','Descrição da Conta'],
                    var_name='data',
                    value_name='valor'
                )
           
                # Tratamento de nulos
                df_long = df_long.assign(valor =lambda x: x['valor'].fillna(0))
                                        
                # Ordenação
                df_long = df_long.sort_values(['Código da Conta','data'])
                
                # Salvar arquivo processado
                output_file = f"{nome}_rotacionado.csv"
                output_path = os.path.join(output_folder, output_file)
                df_long.to_csv(output_path, index=False, encoding='utf-8')
                
                print(f"Arquivo {arquivo} processado com sucesso! Salvo como {output_file}")
                print(df_long.head())
                
            except Exception as e:
                print(f"Erro ao processar o arquivo {arquivo}: {str(e)}")

# Configuração dos caminhos (usando Path para compatibilidade entre sistemas)
base_dir = Path(r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg")
input_folder = base_dir / "_03_ANALISE_EXPLORATORIA\_03_02_analises_01"
output_folder = base_dir / "_04_ETL/_04_01_datasets_pivotados"

# Executar o processamento
process_files(input_folder, output_folder)

In [ ]:
df = pd.read_csv(r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_04_ETL\_04_01_datasets_pivotados\Balanço Patrimonial (BP)_rotacionado.csv")

In [ ]:
df.head(100)  # Exibir as primeiras linhas do DataFrame para verificação

### Alterar cabeçalhos para maiúsculas e substituir espaços por underline

In [ ]:
pasta_origem = r'E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_04_ETL\datasets_pivotados'
pasta_destino = r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_04_ETL\_04_01_dataset_tratado"

for arquivo in os.listdir(pasta_origem):
    if arquivo.endswith('.csv'):
        entrada = os.path.join(pasta_origem, arquivo)
        saida = os.path.join(pasta_destino, arquivo)
        try:
            df = pd.read_csv(entrada, encoding='utf-8')
            df.columns = [unidecode(str(col)).upper().replace(' ', '_') for col in df.columns]
            df.to_csv(saida, index=False)
            print(f"Convertido: {arquivo}")
        except Exception as e:
            print(f"Erro ao converter {arquivo}: {e}")

#### DEIXAR TUDO SEM ACENTO E CAIXA ALTA

In [ ]:
pasta_origem = r'E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_04_ETL\datasets_pivotados'
pasta_destino = r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_04_ETL\_04_02_dataset_tratado"

for arquivo in os.listdir(pasta_origem):
    if arquivo.endswith('.csv'):
        entrada = os.path.join(pasta_origem, arquivo)
        saida = os.path.join(pasta_destino, arquivo)
        try:
            df = pd.read_csv(entrada, encoding='utf-8')

            # Normaliza os nomes das colunas para maiúsculas e sem acentos
            df.columns = [unidecode(str(col).upper()) for col in df.columns]
            df = df.dropna()

            # Converte todos os valores de string para maiúsculas e sem acentos
            for col in df.select_dtypes(include='object').columns:
                df[col] = df[col].apply(
                    lambda val: unidecode(str(val).strip().upper()) if pd.notnull(val) else val
                )

            df.to_csv(saida, index=False, encoding='utf-8')
            print(f"Convertido: {arquivo}")
        except Exception as e:
            print(f"Erro ao converter {arquivo}: {e}")

Convertido: Balanço Patrimonial (BP)_rotacionado.csv
Convertido: Demonstração de Fluxo de Caixa (DFC)_rotacionado.csv
Convertido: Demonstração de Resultados (DRE)_rotacionado.csv
